In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# nematode
## Cao

In [ ]:
cao = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Cao/data.h5")
utils.peek(cao, "build/nematode/Cao")
cao.obs.head()

In [ ]:
cao.obs["cluster"] = "cluster_" + cao.obs["cluster"]
cao.obs.dtypes

In [ ]:
cao_model = cb.directi.fit_DIRECTi(
    cao, cao.uns["seurat_genes"],
    **fixed_model_kwargs
)
cao.latent = cao_model.inference(cao)

In [ ]:
ax = cao.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/nematode/Cao/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = cao.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/nematode/Cao/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = cao.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/nematode/Cao/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = cao.visualize_latent("region", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/nematode/Cao/region.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
cao.write_dataset("build/nematode/Cao/Cao.h5")

In [ ]:
%%capture capio
cao_models = [cao_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    cao_models.append(cb.directi.fit_DIRECTi(
        cao, cao.uns["seurat_genes"],
        **fixed_model_kwargs, 
        random_seed=i
    ))
cao_blast = cb.blast.BLAST(cao_models, cao, eps=0.1)
cao_blast.save("build/nematode/Cao")

In [ ]:
with open("build/nematode/Cao/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/nematode/Cao/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(cao_blast, "build/nematode/Cao")

In [ ]:
%%writefile build/nematode/Cao/predictable.txt
cell_ontology_class
cell_type1
cluster
region